# 1. 데이터 읽어오기

In [35]:
import numpy as np
import tensorflow as tf
import glob
import os
import re

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


In [36]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'


# 2. 데이터 전처리

In [37]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [38]:
# 정제된 문장들 확인
corpus = []

for sentence in raw_corpus:
    # 정제된 문장 체크
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    # 토큰 길이가 15를 초과하면 건너뛴다
    if len(preprocessed_sentence.split()) > 15: continue
    # corpus에 담기
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:10]

['<start> hook <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face verse <end>',
 '<start> somethin ain t right when we talkin <end>']

In [39]:
# 토큰화 함수

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 변환
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, 
                                                      filters=' ',
                                                      oov_token="<unk>")
    # corpus를 이용해 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기 (문장 뒤에 패딩을 붙여줌)
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰춤
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  968    3 ...    0    0    0]
 [   2    4   95 ...   10   12    3]
 [   2   38  133 ...    0    0    0]
 ...
 [   2  202    3 ...    0    0    0]
 [   2  416    9 ...    0    0    0]
 [   2    9 1560 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe18e9e4710>


In [40]:
print(tensor[:3])

[[  2 968   3   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2   4  95 105  58  31 168   5  11 133  23  29  10  12   3]
 [  2  38 133  29  10  12   3   0   0   0   0   0   0   0   0]]


In [41]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [42]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 968   3   0   0   0   0   0   0   0   0   0   0   0]
[968   3   0   0   0   0   0   0   0   0   0   0   0   0]


# 3. Train, Test 분할

In [60]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

print("Source Train:", enc_train.shape , enc_val.shape)
print("Target Train:", dec_train.shape , dec_val.shape)

Source Train: (124981, 14) (31246, 14)
Target Train: (124981, 14) (31246, 14)


In [62]:
# 데이터 셋

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 train 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 4. 모델 생성

In [63]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop = tf.keras.layers.Dropout(0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.drop(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [64]:
# 데이터셋에서 데이터 한 배치만 불러오기 

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.72428646e-04, -1.06003688e-04,  1.39433105e-04, ...,
          4.38288334e-05,  2.51213409e-04, -1.43082012e-04],
        [ 4.47244471e-04,  6.96388888e-05, -6.58977078e-05, ...,
          1.60912183e-04,  6.49145688e-04, -4.10554901e-04],
        [ 4.66923317e-04,  5.29909274e-04, -3.64989392e-04, ...,
          4.59851901e-04,  1.05990062e-03, -6.39486476e-04],
        ...,
        [-5.62848691e-05,  6.30068360e-04,  2.71459256e-04, ...,
         -1.93392579e-03, -1.64934830e-03, -8.85377231e-04],
        [-8.63567751e-04,  3.52301489e-04,  2.56815227e-04, ...,
         -2.46831309e-03, -2.07144674e-03, -7.77114765e-04],
        [-1.63625285e-03,  6.89942462e-05,  2.10978731e-04, ...,
         -2.91815680e-03, -2.43651215e-03, -6.17110054e-04]],

       [[ 2.72428646e-04, -1.06003688e-04,  1.39433105e-04, ...,
          4.38288334e-05,  2.51213409e-04, -1.43082012e-04],
        [ 5.60698099e-04,  5.09262682e-05,  4

In [65]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  20979712  
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
lstm_7 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_3 (Dense)              multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


# 5. 학습

In [66]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),
          epochs=10, batch_size=256)

Epoch 1/10
489/489 [==============================] - 452s 919ms/step - loss: 3.8500 - val_loss: 2.9389
Epoch 2/10
489/489 [==============================] - 449s 919ms/step - loss: 2.8094 - val_loss: 2.6712
Epoch 3/10
489/489 [==============================] - 451s 922ms/step - loss: 2.4644 - val_loss: 2.4868
Epoch 4/10
489/489 [==============================] - 450s 920ms/step - loss: 2.1428 - val_loss: 2.3546
Epoch 5/10
489/489 [==============================] - 452s 924ms/step - loss: 1.8570 - val_loss: 2.2657
Epoch 6/10
489/489 [==============================] - 453s 926ms/step - loss: 1.6186 - val_loss: 2.2096
Epoch 7/10
489/489 [==============================] - 451s 923ms/step - loss: 1.4239 - val_loss: 2.1807
Epoch 8/10
489/489 [==============================] - 452s 925ms/step - loss: 1.2805 - val_loss: 2.1727
Epoch 9/10
489/489 [==============================] - 451s 922ms/step - loss: 1.1717 - val_loss: 2.1795
Epoch 10/10
489/489 [==============================] - 450s 921m

# 6. 텍스트 생성

In [67]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [68]:
generate_text(model, tokenizer, init_sentence="<start> i want", max_len=20)

'<start> i want to love you p . y . t . <end> '

In [69]:
generate_text(model, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s lookin for that home , and i hope he finds it <end> '

In [70]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you can t be pablo if your work ain t sellin <end> '

In [71]:
generate_text(model, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me like nobody <end> '

- batch_size=256, embedding_size=512, hidden_size를 2048가 최선 (val_loss : 2.17 ~ 2.19)
- 중간에 dropout을 통해 overfitting을 그나마 막아준 것으로 보임
- 문장은 그럴듯 하게 나왔으나, 어퍼스트로피(')를 삭제한 것은 에러로 보임.
- `"he"` -> `"he s lookin for that home , and i hope he finds it"` (good)
- 유일하게 `"i want"` 에서 나온 오류?로 `"i want to love you p . y . t ."`  뒤에 이상한 문자들이 나온 이유가 뭔지 확인 필요...
- 학습시간은 대략 75분(...) 좀 더 여유를 갖고 학습을 하자!